In [1]:
import modin.pandas as pd
import os
import re
pd.set_option('display.max_colwidth', None)

In [2]:
annotated_data_dir = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\5. Annotated data\\"
output_dir = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\6. Annotated Data with Job Titles\\Batch 3\\"

In [17]:
df = pd.read_csv(os.path.join(annotated_data_dir, "annotated_data_2018_batch_3_removed_words.csv"))

print(len(df))
df.head()

19224


Unnamed: 0.1  Unnamed: 0  jobPostingId  \
0             0       41874     589566654   
1             1       41875     589565742   
2             2       41876     589522590   
3             3       41877     589516170   
4             4       41878     589515078   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         companyDetails  \
0        {'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany': {'companyResolutionResult': {'entityUrn': 'urn:li:fs_normalized_company:21698', 'name': 'Texas Association of School Boards', 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/company-logo_200_200/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=9c70yr9z-BolQsbgRU1rR5aUcasSfFNFKm2m3YNbbB8', 'expiresAt': 1750291200000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/company-logo_100_100/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=OJDlxdJONL4sCpX6bclMwS6jnQfqivdRWf1Ip6rtyL8', 'expiresAt': 1750291200000, 'height': 100}, {'width': 400, 'fileIdentifyingUrlPathSegment': '400_400/company-logo_400_400/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=nNJ7R0qlOrxpdCxCGpg1eAf1Lvmfjl7mafstJVSZ9CY', 'expiresAt': 1750291200000, 'height': 400}], 'rootUrl': 'https://media.licdn.com/dms/image/v2/D560BAQFPr6OCPnUQiQ/company-logo_'}}, 'type': 'SQUARE_LOGO'}, '$recipeType': 'com.linkedin.voyager.deco.organization.shared.WebCompactCompany', 'universalName': 'tasbpage', 'url': 'https://www.linkedin.com/company/tasbpage'}, 'company': 'urn:li:fs_normalized_company:21698', '$recipeType': 'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany'}}   
1  {'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany': {'companyResolutionResult': {'entityUrn': 'urn:li:fs_normalized_company:165627', 'name': 'Quicken Loans', 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/company-logo_200_200/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=606QVx1QHG22KYbIGa-fR5eoBZdy_XWkv5jGuEHmf7Q', 'expiresAt': 1750291200000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/company-logo_100_100/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=Z4RLM4xVX91ikxsqglRM5YML1CMHAyFWD43Y8bIyeQU', 'expiresAt': 1750291200000, 'height': 100}, {'width': 400, 'fileIdentifyingUrlPathSegment': '400_400/company-logo_400_400/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=qfgWKk-VHMaHPU_QjkX2YWlRLRwwqbEB0VeWcuB8HLA', 'expiresAt': 1750291200000, 'height': 400}], 'rootUrl': 'https://media.licdn.com/dms/image/v2/D560BAQGqZCyxUJsrZg/company-logo_'}}, 'type':

In [18]:
# Define mapping of keywords to roles
role_mapping = {
    'Full-Stack Developer': [r'full.*?stack'],
    'Back-End Developer': [r'back.*?end'],
    'Front-End Developer': [r'front.*?end'],
    'Desktop or Enterprise Applications Developer': [r'desktop', r'enterprise'],
    'Mobile Developer': [r'mobile', r'ios', r'android', r'application', r'app'],
    'DevOps Specialist': [r'dev.*?ops'],
    'System Administrator': [r'system admin', r'systems admin', r'sysadmin'],
    'Database Administrator': [r'database admin', r'dba'],
    'Embedded Applications or Devices Developer': [r'embedded'],
    'Data Scientist or Machine Learning Specialist': [r'data.*?scien', r'machine.*?learning', r'ml', r'ai', r'computer.*?vision', r'nlp'],
    'Data or Business Analyst': [r'analyst'],
    'Data Engineer': [r'data.*?engineer', r'etl', r'elt'],
    'QA or Test Developer': [r'qa', r'test(er)?'],
    'Manager': [r'manager', r'mgr'],
    'Designer': [r'design', r'ui', r'ux'],
    'Researcher': [r'research', r'scientist', r'r&d', r'r & d'],
    'Academic': [r'academic', r'professor', r'lecturer', r'teacher', r'tutor', r'faculty'],
    'Security Professional': [r'security'],
    'Site Reliability Engineer': [r'site reliability engineer', r'sre'],
    'Game or Graphics Developer': [r'game.*?developer', r'game', r'graphics', r'gaming'],
    'Senior Executive (C-Suite, VP, etc.)': [r'chief', r'cto', r'ceo', r'cfo', r'cio', r'vp', r'vice.*?president'],
    'Blockchain Developer': [r'blockchain'],
    'Python Engineer': [r'python'],
    'JavaScript Developer': [r'javascript', r'js'],
    'Java Engineer': [r'java'],
    'Project Manager': [r'pm', r'project manag'],
    'C# Developer': [r'c#'],
    'Node.js Developer': [r'node[.]js'],
    '.Net Developer': [r'dot.*?net', r'[.]net', r'net'],
    'C++ Developer': [r'c[+]+'],
    'SQL Developer': [r'sql'],
    'PHP Developer': [r'php'],
    'Matlab Developer': [r'matlab'],
    'ASP Developer': [r'asp'],
    'TypeScript Developer': [r'typescript'],
    'Django Developer': [r'django'],
    'Rust Developer': [r'rust'],
    'Kotlin Developer': [r'kotlin'],
    'Hadoop Developer': [r'hadoop'],
    'Oracle Developer': [r'oracle'],
    'VBA Developer': [r'vba'],
    'C Developer': [r'c.*?eng', r'c.*?dev', r'c.*?program'],
    'WordPress Developer': [r'wordpress'],
    'Spark Developer': [r'spark'],
    'Ruby Developer': [r'ruby'],
    'React Developer': [r'react'],
    'Drupal Developer': [r'drupal'],
    'Marketing or Sales Professional': [r'marketing', r'sales'],
    ## added myself after this
    'Systems Engineer': [r'system.*?engineer', r'system.*?developer', r'engineer.*?system', r'developer.*?system'], 
    'Network Engineer': [r'network.*?engineer', r'network'],
    'Web Developer': [r'web.*?engineer', r'web.*?developer', r'web'],
    'SharePoint Developer': [r'sharepoint'],
    'BI Developer': [r'business.*?intelligence', r'bi', r'tableau', r'power.*?bi', r'qlikview'],
    'JEE/J2EE Developer': [r'jee', r'j2ee'],
    'Scala Developer': [r'scala'],
    'Support Role': [r'support'],
    'Professional Services': [r'professional.*?services'],
    'SAP Consultant': [r'sap'],
    'Solution Architect': [r'solution.*?architect'],
    'Data Architect': [r'data.*?architect'],
    'Database Developer': [r'database'],
    'Intern': [r'intern'],
    'Graduate Position': [r'graduate'],
    'IT Administrator': [r'it.*?admin'],
    'Software/System/IT Architect': [r'architect'],
    'IT Auditor': [r'audit'],
    'Technician': [r'technician'],
    'IT Engineer': [r'it.*?eng', r'it.*?dev', r'information.*?tech'],
    'GIS Developer': [r'gis'],
    'Cloud Computing Architect': [r'cloud'],
    'Business Consultant': [r'business.*?cons'],
    'SAS Developer': [r'sas'],
    'Customer Service': [r'help.*?desk', r'customer', r'service.*?desk', 'call.*?cent'],
    'Statistical Programmer': [r'statist'],
    'Scrum Master': [r'scrum.*?master'],
    'Tibco Developer': [r'tibco'],
    'Infrastructure Engineer': [r'infrastructure'],
    'Solution Delivery': [r'delivery'],
    'CRM Developer': [r'crm'],
    'Integration Engineer': [r'integration'],
    'Software Developer': [r'software.*?eng', r'eng.*?software', r'develop.*?software', r'software.*?develop', r'sde', r'swe'],
    'Other Consultant Role': [r'consult'],
    'Other Development Role': [r'develop', r'program', r'engineer'],
    'Other IT Role': [r'it', r'information.*?tech'],
    'Other Head/Lead Role': [r'head', r'lead'], 
    'Other Data Role': [r'data']
    
}

# Function to classify titles
def classify_title(title):
    title = title.lower()
    for role, patterns in role_mapping.items():
        for pattern in patterns:
            if re.search(pattern, title):
                return role
    return 'Other'

In [19]:
# Apply the function to the DataFrame
df['role'] = df['title'].apply(classify_title)
df.head()

Unnamed: 0.1  Unnamed: 0  jobPostingId  \
0             0       41874     589566654   
1             1       41875     589565742   
2             2       41876     589522590   
3             3       41877     589516170   
4             4       41878     589515078   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         companyDetails  \
0        {'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany': {'companyResolutionResult': {'entityUrn': 'urn:li:fs_normalized_company:21698', 'name': 'Texas Association of School Boards', 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/company-logo_200_200/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=9c70yr9z-BolQsbgRU1rR5aUcasSfFNFKm2m3YNbbB8', 'expiresAt': 1750291200000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/company-logo_100_100/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=OJDlxdJONL4sCpX6bclMwS6jnQfqivdRWf1Ip6rtyL8', 'expiresAt': 1750291200000, 'height': 100}, {'width': 400, 'fileIdentifyingUrlPathSegment': '400_400/company-logo_400_400/0/1706535671752/tasbpage_logo?e=1750291200&v=beta&t=nNJ7R0qlOrxpdCxCGpg1eAf1Lvmfjl7mafstJVSZ9CY', 'expiresAt': 1750291200000, 'height': 400}], 'rootUrl': 'https://media.licdn.com/dms/image/v2/D560BAQFPr6OCPnUQiQ/company-logo_'}}, 'type': 'SQUARE_LOGO'}, '$recipeType': 'com.linkedin.voyager.deco.organization.shared.WebCompactCompany', 'universalName': 'tasbpage', 'url': 'https://www.linkedin.com/company/tasbpage'}, 'company': 'urn:li:fs_normalized_company:21698', '$recipeType': 'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany'}}   
1  {'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany': {'companyResolutionResult': {'entityUrn': 'urn:li:fs_normalized_company:165627', 'name': 'Quicken Loans', 'logo': {'image': {'com.linkedin.common.VectorImage': {'artifacts': [{'width': 200, 'fileIdentifyingUrlPathSegment': '200_200/company-logo_200_200/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=606QVx1QHG22KYbIGa-fR5eoBZdy_XWkv5jGuEHmf7Q', 'expiresAt': 1750291200000, 'height': 200}, {'width': 100, 'fileIdentifyingUrlPathSegment': '100_100/company-logo_100_100/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=Z4RLM4xVX91ikxsqglRM5YML1CMHAyFWD43Y8bIyeQU', 'expiresAt': 1750291200000, 'height': 100}, {'width': 400, 'fileIdentifyingUrlPathSegment': '400_400/company-logo_400_400/0/1698811195493/quicken_loans_logo?e=1750291200&v=beta&t=qfgWKk-VHMaHPU_QjkX2YWlRLRwwqbEB0VeWcuB8HLA', 'expiresAt': 1750291200000, 'height': 400}], 'rootUrl': 'https://media.licdn.com/dms/image/v2/D560BAQGqZCyxUJsrZg/company-logo_'}}, 'type':

In [20]:
len(df[df['role']=="Other"][['title', 'role']])
# df[df['role']=="Other"][['title', 'role']].to_csv('other_roles.csv')

374

In [21]:
df['role'].value_counts()[:10]

role
Manager                                          2790
Software Developer                               1267
Mobile Developer                                 1072
Data Scientist or Machine Learning Specialist    1036
Data or Business Analyst                          997
C Developer                                       932
Designer                                          735
Project Manager                                   621
.Net Developer                                    567
Java Engineer                                     551
Name: count, dtype: int64

In [22]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df.columns

Index(['jobPostingId', 'companyDetails', 'title', 'description',
       'workRemoteAllowed', 'formattedLocation', 'work_model', 'soc_code',
       'country', 'year', 'Company ID', 'Industry', 'count_masc_words',
       'count_fem_words', 'score', 'gender_category',
       'equal_opportunity_employer', 'masc_pronouns_count',
       'fem_pronouns_count', 'neutral_pronouns_count', 'fem_words_list',
       'masc_words_list', 'role'],
      dtype='object')

In [23]:
len(df)

19224

In [24]:
df.to_csv(os.path.join(output_dir, "annotated_data_2018_batch_3_removed_words.csv"), index=False)